# Synaptic Weights

@[Xiaoyu Chen](mailto:c-xy17@tsinghua.org.cn) 

In a brain model, synaptic weights, the strength of the connection between presynaptic and postsynaptic neurons, are crucial to the dynamics of the model. In this section, we will illutrate how to build synaptic weights in a synapse model.

In [34]:
import brainpy as bp
import brainpy.math as bm
import numpy as np

bp.math.set_platform('cpu')

## Creating Static Weights

Some computational models focus on the network structure and its influence on network dynamics, thus not modeling neural plasticity for simplicity. In this condition, synaptic weights are fixed and do not change in simulation. They can be stored as a scalar, a matrix or a vector depending on the connection strength and density.

### 1. Storing weights with a scalar

If all synaptic weights are designed to be the same, the single weight value can be stored as a scalar in the synpase model to save memory space.

In [2]:
weight = 1.

The `weight` can be stored in a synapse model. When updating the synapse, this `weight` is assigned to all synapses by scalar multiplication. 

### 2. Storing weights with a matrix

**When the synaptic connection is dense and the synapses are assigned with different weights**, weights can be stored in a matrix $W$, where $W(i, j)$ refers to the weight of presynaptic neuron $i$ to postsynaptic neuron $j$.

BrainPy provides `brainpy.training.initialize.Initializer` (or `brainpy.init` for short), the tutorial of which is displayed in [Initializing Connection Weights](../tutorial_training/initializing_connection_weights.ipynb), to initialize synaptic weights as a matrix.

For example, a weight matrix can be constructed using `brainpy.init.Uniform`, which initializes weights with a random distribution:

In [15]:
pre_size = (4, 4)
post_size = (3, 3)

uniform_init = bp.init.Uniform(min_val=0., max_val=1.)
weights = uniform_init((pre_size, post_size))
print('shape of weights: {}'.format(weights.shape))

shape of weights: (16, 9)


Then, the weights can be assigned to the corresponding connection. For example, an all-to-all connection matrix can be obtained as below (for creating synaptic connections, please refer to [Creating Synaptic Connections](synaptic_connections.ipynh)):

In [19]:
conn = bp.conn.All2All()
conn(pre_size, post_size)
conn_mat = conn.requires('conn_mat')  # request the connection matrix

Therefore, `weights[i, j]` refers to the synaptic weight of the connection `(i, j)`.

In [20]:
i, j = (2, 3)
print('whether (i, j) is connected: {}'.format(conn_mat[i, j]))
print('synaptic weights of (i, j): {}'.format(weights[i, j]))

whether (i, j) is connected: True
synaptic weights of (i, j): 0.19848954677581787


### 3. Storing weights with a vector

**When the synaptic connection is sparse, using a matrix to store synaptic weights is too wasteful.** Instead, the weights can be stored in a vector which has the same length as the synaptic connections.

<img src="../_static/synapses_and_weights.png" width="400 px">

Weights can be assigned to the corresponding synapses as long as the they are aligned with each other.

In [28]:
size = 5

conn = bp.conn.One2One()
conn(size, size)
pre_ids, post_ids = conn.requires('pre_ids', 'post_ids')

print('presynaptic neuron ids: {}'.format(pre_ids))
print('postsynaptic neuron ids: {}'.format(post_ids))
print('synapse ids: {}'.format(bm.arange(size)))

presynaptic neuron ids: [0 1 2 3 4]
postsynaptic neuron ids: [0 1 2 3 4]
synapse ids: [0 1 2 3 4]


The weight vector is aligned with the synapse vector, i.e. synapse ids :

In [31]:
weights = bm.random.uniform(0, 2, size)

for i in range(size):
    print('weight of synapse {}: {}'.format(i, weights[i]))

weight of synapse 0: 1.1220934391021729
weight of synapse 1: 1.3576321601867676
weight of synapse 2: 0.12217831611633301
weight of synapse 3: 0.8358919620513916
weight of synapse 4: 1.6964213848114014


#### Conversion from a weight matrix to a weight vector
For users who would like to obtain the weight vector from the weight matrix, they can first build a connection according to the non-zero elements in the weight matrix and then slice the weight matrix according to the connection:

In [39]:
weight_mat = np.array([[1., 1.5, 0., 0.5], [0., 2.5, 0., 0.], [2., 0., 3, 0.]])
print('weight matrix: \n{}'.format(weight_mat))

conn = bp.conn.MatConn(weight_mat)
pre_ids, post_ids = conn.requires('pre_ids', 'post_ids')

weight_vec = weight_mat[pre_ids, post_ids]
print('weight_vector: \n{}'.format(weight_vec))

weight matrix: 
[[1.  1.5 0.  0.5]
 [0.  2.5 0.  0. ]
 [2.  0.  3.  0. ]]
weight_vector: 
[1.  1.5 0.5 2.5 2.  3. ]


```{note}
However, it is not recommended to use this function when the connection is sparse and of a large scale, because generating the weight matrix will take up too much space.
```

## Creating Dynamic Weights

Sometimes users may want to realize neural plasticity in a brain model, which requires the synaptic weights to change during simulation. In this condition, weights should be considered as **variables**, thus defined as `brainpy.math.Variable`. If it is packed in a synapse model, weight updating should be realized in the `update(_t, _dt)` function of the synapse model.

In [40]:
weights = bm.Variable(bm.ones(10))
weights

Variable(DeviceArray([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32))